In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, date
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import StandardScaler

from math import ceil

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

%matplotlib inline


Using TensorFlow backend.


In [2]:
train = pd.read_csv('all/sales_train.csv')
test = pd.read_csv('all/test.csv')
submission = pd.read_csv('all/sample_submission.csv')
items = pd.read_csv('all/items.csv')
item_cats = pd.read_csv('all/item_categories.csv')
shops = pd.read_csv('all/shops.csv')
print("Ok")

Ok


In [3]:
test_shops = test.shop_id.unique()
train = train[train.shop_id.isin(test_shops)]
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]

MAX_BLOCK_NUM = train.date_block_num.max()
MAX_ITEM = len(test_items)
MAX_CAT = len(item_cats)
MAX_YEAR = 3
MAX_MONTH = 4 # 7 8 9 10
MAX_SHOP = len(test_shops)

In [4]:
train = train.set_index('item_id').join(items.set_index('item_id')).drop('item_name', axis=1).reset_index()

train['month'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
train['year'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))

In [5]:
train = train.drop('date', axis=1)
train = train.drop('item_category_id', axis=1)
train = train.groupby(['shop_id', 'item_id', 'date_block_num', 'month', 'year']).sum()
train = train.sort_index()

In [6]:
scaler = StandardScaler()
cnt_scaler = StandardScaler()

scaler.fit(train.item_price.values.reshape(-1, 1))
cnt_scaler.fit(train.item_cnt_day.values.reshape(-1, 1))

train.item_price = scaler.transform(train.item_price.values.reshape(-1, 1))
train.item_cnt_day = cnt_scaler.transform(train.item_cnt_day.values.reshape(-1, 1))

In [7]:
# train.reset_index().groupby(['item_id', 'date_block_num', 'shop_id']).mean()

In [8]:
price = train.reset_index().set_index(['item_id', 'shop_id', 'date_block_num'])
price = price.sort_index()
# price

In [9]:
def convert(date_block):
    date = datetime(2013, 1, 1)
    date += relativedelta(months = date_block)
    return (date.month, date.year)

def closest_date_block(current_day, item_id, shop_id):
    """Find the block_date which is closest to the current_day, given item_id and shop_id. Returns index integer"""
    if (item_id, shop_id) in price.index:
        search_lst = np.array(price.loc[(item_id, shop_id)].index)        
        return search_lst[np.abs(current_day - search_lst).argmin()]
    return -1
                
def closest_price(current_day, item_id, shop_id):
    closest_date = closest_date_block(current_day, item_id, shop_id)
    if closest_date != -1:
        return price.loc[( item_id, shop_id, closest_date )]['item_price']
    return np.nan

def closest_price_lambda(x):
    return closest_price(34, x.item_id, x.shop_id)

In [10]:
assert closest_date_block(18, 30, 5) == 18

In [11]:
# Some simple math to know what date_block_num to start learning
print(convert(6))
print(convert(18))
print(convert(30))

(7, 2013)
(7, 2014)
(7, 2015)


In [13]:
maxlen = 4 # 4 months
step = 1
# 0: train, 1: val, 2:test
sentences = [[],[],[]]
next_chars = [[], []]
BLOCKS = [6, 18, 30]

count = 0

for s in test_shops:
    shop_items = list(train.loc[s].index.get_level_values(0).unique())
#     print(len(shop_items))
    print(count)
    count = count + 1
    for it in shop_items:     
#         print(count)   
        for i_index, i in enumerate(BLOCKS):
            sentence = []
#             start_time = time.time()  
            closest_pc = closest_price(i, it, s) 
            for j in range(maxlen+1):
                if j < maxlen:
                    if (s, it, i+j) in train.index:
                        r = train.loc[(s, it, i + j)].to_dict(orient='list')                    
                        closest_pc = r['item_price'][0]
                        item_cnt_day = r['item_cnt_day'][0]
                        row = {'shop_id': s, 'date_block_num': i+j, 'item_cnt_day': item_cnt_day, 
                               'month': month, 'item_id': it, 'item_price': closest_pc, 'year': year}
                    else:
#                         closest_pc = closest_price(i, it, s)     
                        month, year = convert(i+j)                    
                        row = {'shop_id': s, 'date_block_num': i+j, 'item_cnt_day': 0, 
                               'month': month, 'item_id': it, 'item_price': closest_pc, 'year': year}
                    sentence.append(row)
                    count = count + 1
                elif i_index < 2:   # not in test set
                    next_chars[i_index].append(row)   
            sentences[i_index].append(sentence)
        
#         print("--- %s seconds ---" % (time.time() - start_time)) 

0
31189
60878
102423
127852
150809
183234
208675
240080
283521
330586
366323
414480
447733
480890
520263
553024
586421
620730
650695
686696
731145
764554
790151
822264
855469
890066
919431
960448
993629
1023786
1028575
1066604
1069629
1100614
1135979
1167624
1182757
1218038
1246227
1279720
1306757


In [14]:

# MAX_BLOCK_NUM = train.date_block_num.max()
MAX_ITEM = len(test_items)
MAX_CAT = len(item_cats)
MAX_YEAR = 3
MAX_MONTH = 4 # 7 8 9 10
MAX_SHOP = len(test_shops)

x_train_o = np.array(sentences[0])
x_val_o = np.array(sentences[1])
x_test_o = np.array(sentences[2])
y_train = np.array([x['item_cnt_day'] for x in next_chars[0]])
y_val = np.array([x['item_cnt_day'] for x in next_chars[1]])

length = MAX_SHOP + MAX_ITEM + MAX_MONTH + 1 + 1 + 1

In [26]:
sentences[0][0]

[{'shop_id': 5,
  'date_block_num': 6,
  'item_cnt_day': 0,
  'month': 7,
  'item_id': 30,
  'item_price': -0.2429610371380407,
  'year': 2013},
 {'shop_id': 5,
  'date_block_num': 7,
  'item_cnt_day': 0,
  'month': 8,
  'item_id': 30,
  'item_price': -0.2429610371380407,
  'year': 2013},
 {'shop_id': 5,
  'date_block_num': 8,
  'item_cnt_day': 0,
  'month': 9,
  'item_id': 30,
  'item_price': -0.2429610371380407,
  'year': 2013},
 {'shop_id': 5,
  'date_block_num': 9,
  'item_cnt_day': -0.15018864969036175,
  'month': 9,
  'item_id': 30,
  'item_price': -0.2757597800953987,
  'year': 2013}]

In [15]:
from sklearn import preprocessing

shop_le = preprocessing.LabelEncoder()
shop_le.fit(test_shops)
shop_dm = dict(zip(test_shops, shop_le.transform(test_shops)))

item_le = preprocessing.LabelEncoder()
item_le.fit(test_items)
item_dm = dict(zip(test_items, item_le.transform(test_items)))

month_le = preprocessing.LabelEncoder()
month_le.fit(range(7,11))
month_dm = dict(zip(range(7,11), month_le.transform(range(7,11))))

In [16]:
def vectorize(inp):
    print('Vectorization...')   
    x = np.zeros((len(inp), maxlen, length), dtype=np.float32)
    for i, sentence in enumerate(inp):
        for t, char in enumerate(sentence):            
            x[i][t][ shop_dm[char['shop_id']] ] = 1        
            x[i][t][ MAX_SHOP + item_dm[char['item_id']] ] = 1
            x[i][t][ MAX_SHOP + MAX_ITEM + month_dm[char['month']] ] = 1
            x[i][t][ MAX_SHOP + MAX_ITEM + MAX_MONTH + 1 ] = char['item_price']
            x[i][t][ MAX_SHOP + MAX_ITEM + MAX_MONTH + 1 + 1] = char['item_cnt_day']    
    return x

x_train = vectorize(x_train_o)
x_val = vectorize(x_val_o)
x_test = vectorize(x_test_o)

Vectorization...
Vectorization...
Vectorization...


In [21]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, length)))
model.add(Dense(1, activation='relu'))

optimizer = RMSprop(lr=0.005)
model.compile(loss='mean_squared_error', optimizer=optimizer)

model.fit(x_train, y_train, batch_size=2048, epochs=6)

Build model...
Epoch 1/6
111404/111404 [==============================] - 10s 90us/step - loss: 0.1216
Epoch 2/6
111404/111404 [==============================] - 9s 82us/step - loss: 0.0809
Epoch 3/6
111404/111404 [==============================] - 9s 79us/step - loss: 0.0673
Epoch 4/6
111404/111404 [==============================] - 9s 82us/step - loss: 0.0606
Epoch 5/6
111404/111404 [==============================] - 9s 82us/step - loss: 0.0531
Epoch 6/6
111404/111404 [==============================] - 9s 79us/step - loss: 0.0490


In [18]:
model.fit(x_val, y_val, batch_size=2048, epochs=13)

Epoch 1/13
111404/111404 [==============================] - 10s 89us/step - loss: 0.0115
Epoch 2/13
111404/111404 [==============================] - 9s 81us/step - loss: 0.0090
Epoch 3/13
111404/111404 [==============================] - 9s 81us/step - loss: 0.0079
Epoch 4/13
111404/111404 [==============================] - 9s 81us/step - loss: 0.0074
Epoch 5/13
111404/111404 [==============================] - 9s 82us/step - loss: 0.0061
Epoch 6/13
111404/111404 [==============================] - 9s 79us/step - loss: 0.0055
Epoch 7/13
111404/111404 [==============================] - 9s 81us/step - loss: 0.0057
Epoch 8/13
111404/111404 [==============================] - 9s 81us/step - loss: 0.0045
Epoch 9/13
111404/111404 [==============================] - 9s 80us/step - loss: 0.0056
Epoch 10/13
111404/111404 [==============================] - 9s 80us/step - loss: 0.0048
Epoch 11/13
111404/111404 [==============================] - 9s 81us/step - loss: 0.0042
Epoch 12/13
111404/111404 [==

In [22]:
predict_test = model.predict(x_test)
predict_test = cnt_scaler.inverse_transform(predict_test)

In [23]:

test = pd.read_csv('all/test.csv')
test = test.set_index(['shop_id', 'item_id'])
test['item_cnt_month'] = 0

for index, sentence in enumerate(x_test_o):
    (shop_id, item_id) = (sentence[0]['shop_id'], sentence[0]['item_id'])
    test.loc[(shop_id, item_id)]['item_cnt_month'] = predict_test[index]
    
    

test = test.reset_index().drop(['shop_id', 'item_id'], axis=1)
test.to_csv('submission.csv', index=False)
